In [1]:
import pandas as pd
import numpy as np
#from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.base import clone
#from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



*Expriment objective:*
To determine credit card level by account information for credit card holders. 
*Ethic Considerations:*
Gender is included in this dataset. Though as we can see, it does not contribute too significantly to classification, at least in forest classification, it *is* listed as a binary. 

This data is taken from Kaggle, and prior, was pulled from a website for datasets. Thus, this is not proprietary nor does it contain private data. 


In [2]:
##Data Collection

#Data From : https://www.kaggle.com/sakshigoyal7/credit-card-customers/download
        #info on the dataset : https://www.kaggle.com/sakshigoyal7/credit-card-customers

credit = pd.read_csv("BankChurners.csv")

credit

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


*Data Collection Discussion*
As stated from the expirimental objective section, this data was downloaded from kaggle in the form of a CSV file

In [3]:
##Data Preprocessing

credit = credit.drop("CLIENTNUM", axis=1)
credit_mapping = {'Blue': 1,
                  'Silver': 2,
                  'Gold': 3, 
                  'Platinum': 4}
credit['Card_Category'] = credit['Card_Category'].map(credit_mapping)
credit = pd.get_dummies(credit, drop_first=True)

credit_all = credit

y = credit['Card_Category']
X = credit.drop("Card_Category", axis = 1)



X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)



stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

*Data Preprocessing*
As there are four classes, I chose to manually encode the class lables to numbers - as the classes are indeed ordinal, I thought this was a good was to encode them.

I have also dropped CLIENTNUM, as it is essentially an index, and unnecessary to examine. 
There were no other missing or obselete variables, thus I initiated panda's get_dummies to encode the categorical variables present such as Gender and Marital_status.

I have chose to scale the data by StandardScaler as the three models I chose need a scaled dataset to process. 

For feature selection, please see "#Model Comparison with chosen features" section. I have used a random forrest to determine feature importance, and select features that had an importance of over 0.03 - resulting in 12 features. I have chosen to compare logistic regression, SVM, and KNN both with all of the features, and with only the selected features over the random forest threshold. 

In [6]:
y

0        1
1        1
2        1
3        1
4        1
        ..
10122    1
10123    1
10124    1
10125    1
10126    2
Name: Card_Category, Length: 10127, dtype: int64

In [4]:
##Model Comparison with all features

#Logistic Regression
lr = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', multi_class='ovr', max_iter = 2000)
lr.fit(X_train_std, y_train)
print('Training accuracy LR all features:', lr.score(X_train_std, y_train))
print('Test accuracy LR all features:', lr.score(X_test_std, y_test))
print("")


#Suport Vector Machine 
svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train_std, y_train)
print('Training accuracy SVM all features:', svm.score(X_train_std, y_train))
print('Test accuracy SVM all features:',  svm.score(X_test_std, y_test))
print("")


##KNN 
knn = KNeighborsClassifier(n_neighbors=5, 
                           p=2, 
                           metric='minkowski')
knn.fit(X_train_std, y_train)
knn.score(X_train_std, y_train)
print('Training accuracy KNN all features:', knn.score(X_train_std, y_train))
print('Test accuracy KNN all features:', knn.score(X_test_std, y_test))
print("")




Training accuracy LR all features: 0.9659988713318285
Test accuracy LR all features: 0.9647910496873972

Training accuracy SVM all features: 0.9688205417607223
Test accuracy SVM all features: 0.9657782165185916



NameError: name 'KNeighborsClassifier' is not defined

In [91]:
##Model Comparison with chosen features


#Random forest to asses feature importance 
feat_labels = credit.columns[1:]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=0)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

######Paired-down features

sfm = SelectFromModel(forest, threshold=0.03, prefit=True)
X_selected = sfm.transform(X_train)

print("")
print('Number of features that meet threshold criterion of 0.03:', 
      X_selected.shape[1])
feat_list_subset = []
for f in range(X_selected.shape[1]):
    feat_list_subset.append(feat_labels[indices[f]])
credit2 = credit[feat_list_subset]

print(feat_list_subset)

X = credit2.iloc[:, :]


X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)


#LR with chosen features
lr = LogisticRegression(penalty='l1', C=1.0, solver='liblinear', multi_class='ovr', max_iter = 2000)
lr.fit(X_train_std, y_train)
print("")
print('Training accuracy LR paired features:', lr.score(X_train_std, y_train))
print('Test accuracy LR paired features:', lr.score(X_test_std, y_test))
print("")



#SVM with chose features
svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train_std, y_train)
print('Training accuracy SVM paired features:', svm.score(X_train_std, y_train))
print('Test accuracy SVM paired features:', svm.score(X_test_std, y_test))
print("")



##KNN with chosen features
knn = KNeighborsClassifier(n_neighbors=5, 
                           p=2, 
                           metric='minkowski')
knn.fit(X_train_std, y_train)
knn.score(X_train_std, y_train)
print('Training accuracy KNN paired features:', knn.score(X_train_std, y_train))
print('Test accuracy KNN paired features:', knn.score(X_test_std, y_test))



 1) Credit_Limit                   0.190710
 2) Avg_Open_To_Buy                0.177840
 3) Avg_Utilization_Ratio          0.053073
 4) Income_Category_Less than $40K 0.052178
 5) Total_Trans_Amt                0.050517
 6) Total_Trans_Ct                 0.040455
 7) Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1 0.035818
 8) Total_Ct_Chng_Q4_Q1            0.034511
 9) Total_Revolving_Bal            0.034299
10) Total_Amt_Chng_Q4_Q1           0.032922
11) Income_Category_$40K - $60K    0.032184
12) Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2 0.031316
13) Dependent_count                0.029420
14) Months_on_book                 0.027534
15) Income_Category_$60K - $80K    0.025961
16) Gender_M                       0.018836
17) Card_Category                  0.017743
18) Total_Relationship_Count       0.017440
19) Inco

*Model Comparison*

Training accuracy LR all features: 0.9659988713318285
Test accuracy LR all features: 0.9647910496873972

Training accuracy SVM all features: 0.9688205417607223
Test accuracy SVM all features: 0.9657782165185916

Training accuracy KNN all features: 0.9442720090293454
Test accuracy KNN all features: 0.9361632115827575

------------------------------------------------------------------------

Training accuracy LR paired features: 0.9503386004514672
Test accuracy LR paired features: 0.9460348798947023

Training accuracy SVM paired features: 0.9533013544018059
Test accuracy SVM paired features: 0.9473511023362948

Training accuracy KNN paired features: 0.9585214446952596
Test accuracy KNN paired features: 0.935176044751563




Between the models, Support Vector Machine with all of the features functioned the best. Too, between the feature-scaled models, SVM was most accurate as well. 

*Experiment Summary*
The data for this project was collected from Kaggle's project Bank Churners. To preprocess the data, I had to map class lables to oridinal values, and encode the rest of the categorical variables via Panda's get_dummies. The data was scaled via StandardScaler, and did feature selection via random forest feature importances. Overall, the SVM model performed best the all-feature data, and KNN performed best with the paired-feature data. 

Overall, it was very interesting to see which feature impacted the classification of credit card level most- including transaction amount and credit limit which make sense, but also features such as Avg open to buy credit line, which, at face value, doesn't seem like it would impact the classification as much as it does (second highest impact). The SVM model was able to pretty well classify the data into card categories, as were the other models. 